<a href="https://colab.research.google.com/github/Brazza32/ADS2002-Catheter/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import cv2

## When calling in massive, call the correct folder path for the model you a

image_folder = '/content/drive/MyDrive/Catheter - ADS2002/Sample Images'
images = []
studyid = []

for filename in os.listdir(image_folder):
  img = cv2.resize(cv2.imread(os.path.join(image_folder, filename),cv2.IMREAD_GRAYSCALE),(2028,2020))
  img = img/255
  if img is not None:
    images.append(img)

for filename in os.listdir(image_folder):
  studyid.append(filename )

new = pd.DataFrame()
new['StudyInstanceUID'] = studyid
new['ImageData'] = images

In [4]:
catheter = pd.read_csv("/content/drive/MyDrive/Catheter - ADS2002/ranzcr-clip-catheter-line-classification/train.csv")
catheter.drop_duplicates() #No duplicates (same number of rows)
train = pd.read_csv("/content/drive/MyDrive/Catheter - ADS2002/ranzcr-clip-catheter-line-classification/train.csv")
train_annotations = pd.read_csv("/content/drive/MyDrive/Catheter - ADS2002/ranzcr-clip-catheter-line-classification/train_annotations.csv")
merged = pd.merge(train, train_annotations, on = "StudyInstanceUID",how = "inner")
merged['StudyInstanceUID'] = merged['StudyInstanceUID'] + '.jpg'

In [5]:
BinaryETT = merged.copy()

BinaryETT['ETT'] = BinaryETT['ETT - Normal']
BinaryETT['ETT'] = BinaryETT['ETT'] + BinaryETT['ETT - Abnormal']
BinaryETT['ETT'] = BinaryETT['ETT'] + BinaryETT['ETT - Borderline']

BinaryETT.drop(BinaryETT[(BinaryETT['ETT'] == 2)].index, inplace=True)
BinaryETT.drop(BinaryETT[(BinaryETT['ETT'] == 3)].index, inplace=True)

BinaryETT = BinaryETT[['ETT','StudyInstanceUID']]

ETTData = pd.merge(new, BinaryETT, on = "StudyInstanceUID",how = "inner")


In [6]:
BinaryCVC = merged.copy()

BinaryCVC['CVC'] = BinaryCVC['CVC - Normal']
BinaryCVC['CVC'] = BinaryCVC['CVC'] + BinaryCVC['CVC - Abnormal']
BinaryCVC['CVC'] = BinaryCVC['CVC'] + BinaryCVC['CVC - Borderline']

BinaryCVC.drop(BinaryCVC[(BinaryCVC['CVC'] == 2)].index, inplace=True)
BinaryCVC.drop(BinaryCVC[(BinaryCVC['CVC'] == 3)].index, inplace=True)


BinaryCVC = BinaryCVC[['CVC','StudyInstanceUID', 'PatientID','label','data']]

In [7]:
BinaryNGT = merged.copy()

BinaryNGT['NGT'] = BinaryNGT['NGT - Normal']
BinaryNGT['NGT'] = BinaryNGT['NGT'] + BinaryNGT['NGT - Abnormal']
BinaryNGT['NGT'] = BinaryNGT['NGT'] + BinaryNGT['NGT - Borderline']

BinaryNGT.drop(BinaryNGT[(BinaryNGT['NGT'] == 2)].index, inplace=True)
BinaryNGT.drop(BinaryNGT[(BinaryNGT['NGT'] == 3)].index, inplace=True)


BinaryNGT = BinaryNGT[['NGT','StudyInstanceUID', 'PatientID','label','data']]

In [8]:
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(2020,2028,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10))


In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

X = ETTData['ImageData']
Y = ETTData['ETT']

X = np.stack(X)
Y = np.stack(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [ ]:
ettclasif = model.fit(X_train, Y_train, epochs=10,
                    validation_data=(X_test, Y_test))

Epoch 1/10
